## Import

In [1]:
from config import car_parameter_fname
from config import reference_parameter_fname

import sys
sys.path.append('../')
from subword_df_handler import SubwordDocumentFrequencyHandler
from subword_df_handler import extract_positive_words
from subword_df_handler import pprint_word_list

## Step 1. Extract positive subwords

In [3]:
car_subword_handler = SubwordDocumentFrequencyHandler(car_parameter_fname)
car_statistics = car_subword_handler.get_df_distribution_statistics_of_all_words()

ref_subword_handler = SubwordDocumentFrequencyHandler(reference_parameter_fname)
ref_statistics = ref_subword_handler.get_df_distribution_statistics_of_all_words()
car_positive, ref_positive, car_filtered = extract_positive_words(car_statistics, ref_statistics)
print(len(car_positive), len(ref_positive), len(car_filtered))

 computing (std/mean, mean, max/mean, argmax) was done.        
 computing (std/mean, mean, max/mean, argmax) was done.        
8979 19090 3614


In [4]:
for word in '엔진 엔진오일 차량 차대 중고 시동 배터리 유리'.split():
    print(word, word in car_positive, word in car_filtered)

엔진 True True
엔진오일 True True
차량 True True
차대 True True
중고 True True
시동 True True
배터리 True True
유리 True False


In [5]:
pprint_word_list(list(car_filtered.keys()))

     HUD	     카이엔	      시흥	    FULL	     교환후
     작업중	   거래되고있	     rea	    뉴SM7	     봉고3
     저소음	      경차	   신형에쿠스	      ve	      폴딩
      내수	     비쥬얼	    흠잡을곳	      팔린	   하이브리드
     그랜저	      강성	     적용된	     논현동	     오디오
     디젤이	    보입니다	     106	      본넷	     아이폰
   싼타페CM	      탈거	     향균필	     설정하	   뉴임프레션
     편의장	      차문	      차급	     가격인	   궁금하신점
     많지않	    블랙박스	     작업하	     확인후	      ad
     시세표	   중고타이어	   프레스티지	     탈거하	    인터쿨러
     CAR	    중고시세	     벤투스	     와이퍼	     뉴아반
   오토갤러리	    출력부족	      라구	      ce	      히터
   완전무사고	     마감재	    만족하실	      레조	     770
    판매했다	     버튼들	      i3	     차세대	     GPS
   디스커버리	     대리점	      검수	      토탈	     엔진에
     잡소리	      상용	     차라는	   title	     튜닝카
     430	     천안시	    주행능력	    무료출장	      트렉
   BMW52	     세이프	      정차	     사제휠	    휠하우스
     출고하	      C2	      레저	     광고비	     룸미러
     제시되	      마세	     드릴수	    서스펜션	      가좌
      FX	   헤드라이트	     뉴클릭	   알아봅시다	     라구나
      네바	     레이휠	     TEL	     887	     일산동
    기아에서	 

## Step 2. Extract category - sensitive subwords

In [6]:
from specific_word_extractor import extract_category_sensitive_words
sensitive_words_by_category = extract_category_sensitive_words(car_subword_handler,
                                                               car_statistics,
                                                               pos_min_df_nstd=3,
                                                               pos_min_of_df_mean_ratio=4
                                                              )
len(sensitive_words_by_category)

27

In [7]:
from utils import read_list
from utils import remove_alphabet_number_comb
index2category = read_list('./car_index.txt')

for c, words in enumerate(sensitive_words_by_category):
    print('{}\ncategory ({}), {}'.format('-'*50,c, index2category[c]))
    pprint_word_list(remove_alphabet_number_comb(words))

--------------------------------------------------
category (0), A6
 bigfile	   아이패드3	    로코모티	   부육셰히르	    미사강변
   66GHz	     A6칩	   10GHz	    byte	   아이폰5에
   60GHz	     아라우	    입주예정	     A6에	  아우디뉴A6
    A6블록	     로스토	     홈버튼	   30GHz	   70GHz
 제조아우디차명	   아이패드2	   40GHz	   80GHz	   광명역세권
   50GHz	   우니베르시	 아우디A6중고	      하그	   00GHz
     운정역	    가람마을	    A6차량	    쿼드코어	   아이폰4S
      Vf	   뉴아이패드	   아이팟터치	 Tallinn	    RegN
     레티나	      알메	     크레테	     출시국	     A6블
    드니프로	     코어i	   아이폰5C	   20GHz	 아우디A6차량
     Plo	 Sportiv	    A6모델	  Olimpi	    에스파뇰
--------------------------------------------------
category (1), BMW5
 원동기형식N5	     d가격	   님펜부르크	     마리엔	   F10가격
 뉴5시리즈모델	 차명BMW52	   차명BMW	    welt	 장외주식114
  원동기형식N	    DTM에	     코아스	 제조BMW차명	 차대번호WBA
    와인에셋	 비엠더블류네비	     비머몰	  우수중소기업	    휠코리아
 BMW바바리안	    komm	   투자컨설팅
--------------------------------------------------
category (2), BMW
   님펜부르크	     마리엔	    welt	    DTM에	   문학사상사
 제조BMW차명	     비머몰	    휠코리아	 BMW바바리안	    komm
-

## Step 3. Extract category - specific subwords

In [8]:
from config import car_corpus_directory
from collections import defaultdict
import datetime
import os
import time
from scipy.sparse import csr_matrix
from scipy.io import mmwrite

from specific_word_extractor import calculate_cooccurrence

In [9]:
corpus_directory = car_corpus_directory
D_positive = car_filtered
subword2index = car_subword_handler.subword2index

calculate_cooccurrence(sensitive_words_by_category,
                       D_positive,
                       subword2index,
                       corpus_directory,
                       './',
                       debug=True)

calculating cooccurrence in 0 / 27 corpus (0:00:01.894959)
calculating cooccurrence in 1 / 27 corpus (0:00:01.432367)
calculating cooccurrence in 2 / 27 corpus (0:00:00.119239)
